# Image segmentation <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI` environment only.
This is due to the package `rsgislib` not currently being installed on the DEA Sandbox. `rsgislib` cannot be installed on the DEA Sandbox until `conda` is implemented.
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)

## Background
In the last two decades, as the spatial resolution of satellite images has increased, remote sensing has begun to shift from a focus on pixel-based analysis towards Geographic Object-Based Image Analysis (GEOBIA), which aims to group pixels together into meaningful image-objects. 
There are two advantages to a GEOBIA worklow; one, we can reduce the 'salt and pepper' effect typical of classifying pixels; and two, we can increase the computational efficiency of our workflow by grouping pixels into fewer, larger, but meaningful objects. 
A review of the emerging trends in GEOBIA can be found in [Chen et al. (2017)](https://www.tandfonline.com/doi/abs/10.1080/15481603.2018.1426092).

## Description
This notebook demonstrates two methods for conducting `image segmentation`, which is a common image analysis technique used to transform a digital satellite image into objects. 
In brief, [image segmentation](https://en.wikipedia.org/wiki/Image_segmentation) aims to partition an image into segments, where each segment consists of a group of pixels with similar characteristics. 
A number of algorithms exist to perform image segmentation, two of which are shown here:

1. [Quickshift](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.quickshift), implemented through the python package `scikit-image`
2. [Shepherd Segmentation](https://www.mdpi.com/2072-4292/11/6/658), implemented through the package `rsgislib`

**Note:** Image segmentation at very large scales can be both time and memory intensive, and the examples shown here will become prohibitively time consuming at scale. 
The notebook [Tiled, Parallel Image Segmentation](https://github.com/GeoscienceAustralia/dea-notebooks/blob/chad/segmentation/parallel_segmentation.ipynb) builds upon the image segmentation algorithm developed by [Shepherd et al. (2019)](https://www.mdpi.com/2072-4292/11/6/658) to run image segmentation across multiple cpus.

***

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

In [3]:
%matplotlib inline

import sys
import datacube
import xarray as xr
import numpy as np
import scipy
import matplotlib.pyplot as plt
from osgeo import gdal
from datacube.helpers import write_geotiff
from skimage.segmentation import quickshift
#from rsgislib.segmentation import segutils

sys.path.append('../Scripts')
from deafrica_datahandling import array_to_geotiff
from deafrica_datahandling import load_ard, mostcommon_crs
from deafrica_plotting import rgb, display_map
from deafrica_bandindices import calculate_indices


### Connect to the datacube

In [4]:
dc = datacube.Datacube(app='Image_segmentation')


## Load Sentinel 2 data from the datacube

Here we are loading in a timeseries of `Sentinel 2` satellite images through the datacube API using the [load_ard](https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Frequently_used_code/Using_load_ard.ipynb) function. This will provide us with some data to work with.

In [5]:
lat, lon =  6.5, 1.4
#lat, lon =  -1.4, 6.5
buffer = 0.075

x = (lon-buffer, lon+buffer)
y =  (lat+buffer, lat-buffer)

# Create a reusable query
query = {
    'x': x,
    'y': y,
    'time': ('2018-01', '2018-06'),
    #'time': ('2018-01', '2018-12'),
    'resolution': (-30, 30)
}

In [6]:
display_map(x=x, y=y)

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [7]:
output_crs = mostcommon_crs(dc=dc, product='s2a_msil2a', query=query)

# Load available data from all three Landsat satellites
ds = load_ard(dc=dc, 
              products=['s2a_msil2a', 's2b_msil2a'],
                group_by='solar_day', output_crs=output_crs, 
              **query)

# Print output data
print(ds)

Using pixel quality parameters for Sentinel 2
Finding datasets
    s2a_msil2a
    s2b_msil2a
Applying pixel quality/cloud mask
Loading 32 time steps
<xarray.Dataset>
Dimensions:          (time: 32, x: 556, y: 555)
Coordinates:
  * time             (time) datetime64[ns] 2018-01-04T10:23:01.016000 ... 2018-06-28T10:19:59.096000
  * y                (y) float64 7.271e+05 7.27e+05 ... 7.105e+05 7.104e+05
  * x                (x) float64 3.147e+05 3.148e+05 ... 3.314e+05 3.314e+05
Data variables:
    coastal_aerosol  (time, y, x) float32 1066.0 1065.0 1065.0 ... nan nan nan
    blue             (time, y, x) float32 1219.0 1216.0 1194.0 ... nan nan nan
    green            (time, y, x) float32 1357.0 1355.0 1355.0 ... nan nan nan
    red              (time, y, x) float32 1231.0 1228.0 1211.0 ... nan nan nan
    red_edge_1       (time, y, x) float32 1588.0 1584.0 1559.0 ... nan nan nan
    red_edge_2       (time, y, x) float32 2401.0 2346.0 2519.0 ... nan nan nan
    red_edge_3       (time, y

## Combine observations into a noise-free statistical summary image
Individual remote sensing images can be affected by noisy and incomplete data (e.g. due to clouds). To produce cleaner images that we can feed into the image segmentation algorithms, we can create summary images, or composites, that combine multiple images into one image to reveal the 'typical' appearance of the landscape for a certain time period.  In the code below, we take the noisy, incomplete satellite images we just loaded and calculate the mean Normalised Difference Vegetation Index (NDVI).  The mean NDVI will be our input into the segmentation algorithms. We will write the NDVI composite to a geotiff, as the Shepherd Segmentation runs on disk.

### Calculate mean NDVI

In [8]:
# First we calculate NDVI on each image in the timeseries
ndvi = calculate_indices(ds, index='NDVI', collection='s2')

# For each pixel, calculate the mean NDVI throughout the whole timeseries
ndvi = ndvi.mean(dim='time', keep_attrs=True)

# Plot the results to inspect
ndvi.NDVI.plot(vmin=0.1, vmax=1.0, cmap='gist_earth_r', figsize=(10, 10))


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-c5abdda80649>", line 2, in <module>
    ndvi = calculate_indices(ds, index='NDVI', collection='s2')
  File "../Scripts/deafrica_bandindices.py", line 313, in calculate_indices
    raise ValueError(f"'{collection}' is not a valid option for "
ValueError: 's2' is not a valid option for `collection`. Please specify either 
'c1' or 'c2'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lo

ValueError: 's2' is not a valid option for `collection`. Please specify either 
'c1' or 'c2'

## Quickshift Segmentation

Using the function [quickshift](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.quickshift) from the python package `scikit-image`, we will conduct an image segmentation on the mean NDVI array.  We then calculate a zonal mean across each segment using the input dataset. Our last step is to export our results as a GeoTIFF.

Follow the quickshift hyperlink above to see the input parameters to the algorithm, and the following [link](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html) for an explanation of quickshift and other segmentation algorithms in `scikit-image`.

In [ ]:
# Convert our mean NDVI xarray into a numpy array, we need
# to be explicit about the datatype to satisfy quickshift
input_array = ndvi.NDVI.values.astype(np.float64)


In [ ]:
# Calculate the segments
segments = quickshift(input_array,
                      kernel_size=5,
                      convert2lab=False,
                      max_dist=10,
                      ratio=1.0)


In [ ]:
# Calculate the zonal mean NDVI across the segments
segments_zonal_mean_qs = scipy.ndimage.mean(input=input_array,
                                            labels=segments,
                                            index=segments)


In [ ]:
# Plot to see result
plt.figure(figsize=(10,10))
plt.imshow(segments_zonal_mean_qs, cmap='gist_earth_r', vmin=0.1, vmax=1.0)
plt.colorbar(shrink=0.9)


### Export result to GeoTIFF
See this [notebook](../Frequently_used_code/Exporting_GeoTIFFs.ipynb) for more info on writing GeoTIFFs to file.

In [ ]:
transform = ds.geobox.transform.to_gdal()
projection = ds.geobox.crs.wkt

# Export the array
array_to_geotiff('segmented_meanNDVI_QS.tif',
                  segments_zonal_mean_qs,
                  geo_transform=transform,
                  projection=projection,
                  nodata_val=np.nan)


## Shepherd Segmentation
Here we conduct an image segmentation on the mean NDVI array using the [runShepherdSegmentation](https://www.rsgislib.org/rsgislib_segmentation.html) function from `rgislib`. 
This scalable segmentation algorithm is seeded using k-means clustering, and can enforce a minimum mapping unit size through an iterative clumping and elimination process [(Shepherd et al. 2019)](https://www.mdpi.com/2072-4292/11/6/658/htm).
This function will output a `.kea` file containing the segmented image, along with a segmented GeoTIFF where the segments are attributed with the zonal mean of the input GeoTIFF (in this case, NDVI). 

To better understand the parameters of the `runShepheredSegmentation` algorithm, read [here](https://www.rsgislib.org/rsgislib_segmentation.html).

The cell directly below sets up the inputs to the [runShepherdSegmentation](https://www.rsgislib.org/rsgislib_segmentation.html) function:


In [ ]:
# Name of the GeoTIFF to export then and segment
tiff_to_segment = 'meanNDVI.tif' 

# Name of the .kea file the GeoTIFF will be converted too
kea_file = 'meanNDVI.kea'

# Name of the segmented .kea file that will be output 
segmented_kea_file = 'meanNDVI_segmented.kea'

# Name of the segmented GeoTIFF attributed with the zonal mean of input file
segments_zonal_mean = 'segments_zonal_mean_shepherdSeg.tif'


We then write out our mean NDVI dataset to a GeoTIFF, and convert the GeoTIFF to a `.kea` file using `gdal.Translate` so it can be read by the `runShepherdSegmentation` function (the `.kea` file format provides a full implementation of the GDAL data model and is implemented within a HDF5 file):

In [ ]:
# Write the mean NDVI dataset to be written to file as a GeoTIFF
write_geotiff(filename=tiff_to_segment, dataset=ndvi[['NDVI']])

# Convert the GeoTIFF into a KEA file format
gdal.Translate(destName=kea_file,
               srcDS=tiff_to_segment,
               format='KEA',
               outputSRS='EPSG:3577')


We can then perform the segmentation on the `.kea` file:

In [ ]:
# Run the image segmentation
segutils.runShepherdSegmentation(inputImg=kea_file,
                                 outputClumps=segmented_kea_file,
                                 outputMeanImg=segments_zonal_mean,
                                 numClusters=20,
                                 minPxls=200)


In [ ]:
# Open and plot the segments attributed with zonal mean NDVI
result = xr.open_rasterio(segments_zonal_mean)
result.plot(vmin=0.1, vmax=1.0, figsize=(10, 10), cmap='gist_earth_r')


Note that we can also open the output `segmented_kea_file` `.kea` file using [xarray.open_rasterio](http://xarray.pydata.org/en/stable/generated/xarray.open_rasterio.html) to view the raw unattributed segments:

In [ ]:
xr.open_rasterio(segmented_kea_file).plot()

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** December 2019

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)